In [ ]:
# A100 환경 및 4비트 양자화를 위한 최신 패키지 설치
!pip install -q -U bitsandbytes transformers accelerate peft datasets trl

############################################################
# 0. Hugging Face 인증
############################################################
from huggingface_hub import login

HF_TOKEN = "your_key"
login(token=HF_TOKEN)

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# ==========================================
# 1. 데이터셋 및 모델 다운로드 (Setup)
# ==========================================
model_id = "meta-llama/Meta-Llama-3-8B"
# 샘플 데이터셋 (영어 명언 데이터)
dataset = load_dataset("Abirate/english_quotes", split="train[:500]")

# 4비트 양자화 설정 (A100에서 속도와 효율 극대화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # A100은 bf16 권장
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 2. Base 모델 로드
print("\n[Step 2] Base 모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 질문 비교를 위한 테스트 함수 ---
def run_comparison_test(stage_name, questions):
    print(f"\n{'='*30}\n {stage_name} 테스트\n{'='*30}")
    model.eval()
    for i, q in enumerate(questions):
        prompt = f"Quote: {q}\nAuthor:"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=30,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"질문 {i+1}: {q}")
        print(f"답변:\n{response}")
        print("-" * 50)

# 테스트 질문 리스트
test_prompts = [
    "Be yourself; everyone else is already taken.",
    "So many books, so little time.",
    "A room without books is like a body without a soul."
]

# 파인튜닝 전 테스트 실행
run_comparison_test("파인튜닝 전 (Base)", test_prompts)



[Step 2] Base 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


 파인튜닝 전 (Base) 테스트
질문 1: Be yourself; everyone else is already taken.
답변:
Quote: Be yourself; everyone else is already taken.
Author: Oscar Wilde
A Quote by Oscar Wilde
A Quote by Oscar Wilde
A Quote by Oscar Wilde
A Quote by Oscar Wilde
A Quote by
--------------------------------------------------
질문 2: So many books, so little time.
답변:
Quote: So many books, so little time.
Author: Maddy L
Book: The Hunger Games
Favorite Quote: “It is the year 2424. I am the last human being alive,
--------------------------------------------------
질문 3: A room without books is like a body without a soul.
답변:
Quote: A room without books is like a body without a soul.
Author: Marcus Tullius Cicero
Title: De Oratore
Genre: Politics
Tags: Philosophy
Read: 6/27/11

--------------------------------------------------


In [ ]:
# Finetuning model

from trl import SFTTrainer
from transformers import TrainingArguments # SFTConfig 대신 TrainingArguments 임포트

# 1. 모델 학습 준비 (기존과 동일)
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 데이터셋 컬럼 'quote'를 'text'로 변경합니다. (SFTTrainer가 'text'를 기본적으로 찾음)
# 이미 'text' 컬럼이 있는 경우 ValueError를 방지하기 위해 조건부로 실행합니다.
if "quote" in dataset.column_names:
    dataset = dataset.rename_column("quote", "text")

# 2. TrainingArguments 생성 (핵심 수정 부분)
training_args = TrainingArguments(
    output_dir="./llama3_slm_final",
    per_device_train_batch_size=8,   # A100 GPU 권장 배치 사이즈
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,              # 모델 성능 향상을 위해 훈련 에포크 증가
    bf16=True,                       # A100 가속 활성화
    logging_steps=10,
    report_to="none",
    remove_unused_columns=False      # 데이터셋 컬럼 유지를 위해 필수
)

# 3. Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args               # 수정된 config 객체 전달
)

print("\n[Step 3] 파인튜닝 시작...")
trainer.train()

# 4. 파인튜닝 후 테스트 실행
run_comparison_test("파인튜닝 후 (Fine-tuned)", test_prompts)

# 5. 파인튜닝된 어댑터 모델 저장
output_dir = "./final_llama3_adapter"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"\n파인튜닝된 모델 어댑터와 토크나이저가 '{output_dir}'에 저장되었습니다.")


[Step 3] 파인튜닝 시작...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.161800
20,0.906800
30,0.884200
40,0.656400
50,0.625100
60,0.595800
70,0.430000
80,0.420700
90,0.502600



 파인튜닝 후 (Fine-tuned) 테스트
질문 1: Be yourself; everyone else is already taken.
답변:
Quote: Be yourself; everyone else is already taken.
Author: Oscar Wilde
--------------------------------------------------
질문 2: So many books, so little time.
답변:
Quote: So many books, so little time.
Author: J.K. Rowling
--------------------------------------------------
질문 3: A room without books is like a body without a soul.
답변:
Quote: A room without books is like a body without a soul.
Author: Marcus Tullius Cicero
--------------------------------------------------

파인튜닝된 모델 어댑터와 토크나이저가 './final_llama3_adapter'에 저장되었습니다.


한국어 테스트

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 1. 모델 및 한국어 데이터셋 설정
model_id = "meta-llama/Meta-Llama-3-8B"
# 한국어 위키데이터 QA 데이터셋 (샘플 1000개 추출)
dataset = load_dataset("maywell/ko_wikidata_QA", split="train[:500]")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # A100 최적화
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2. 모델 로드 (Base)
print("\n[Step 2] Base 모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 한국어 질문 비교를 위한 테스트 함수 ---
def run_ko_test(stage_name, questions):
    print(f"\n{'='*30}\n {stage_name} 테스트\n{'='*30}")
    model.eval()
    for i, q in enumerate(questions):
        # 데이터셋의 'instruction'과 'output' 형식을 고려한 프롬프트
        prompt = f"질문: {q}\n답변:"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                repetition_penalty=1.2 # 한국어 반복 방지
            )

        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        print(f"테스트 {i+1}. 질문: {q}")
        print(f"결과:\n{response}")
        print("-" * 50)

# 한국어 테스트 질문 3개
ko_questions = [
    "대한민국의 수도는 어디인가요?",
    "세종대왕이 만든 문자의 이름은 무엇인가요?",
    "태양계에서 가장 큰 행성은 무엇인가요?"
]

# 파인튜닝 전 테스트
run_ko_test("파인튜닝 전 (Base)", ko_questions)

# 3. 한국어 데이터 파인튜닝 (LoRA)
print("\n[Step 3] 한국어 데이터 파인튜닝 시작...")
model.train()
model = prepare_model_for_kbit_training(model)

# 한국어 QA 데이터셋의 'instruction'과 'output' 컬럼을 'text' 컬럼으로 변환
def format_korean_qa_dataset(example):
    # 'instruction'과 'output' 컬럼을 결합하여 'text' 컬럼 생성
    example['text'] = f"질문: {example['instruction']}\n답변: {example['output']}"
    return example

# 데이터셋에 'text' 컬럼 생성
dataset = dataset.map(format_korean_qa_dataset)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 2. TrainingArguments 생성 (핵심 수정 부분)
training_args = TrainingArguments(
    output_dir="./llama3_korean_slm_final", # 한국어 모델용 출력 디렉토리 변경
    per_device_train_batch_size=8,   # A100 GPU 권장 배치 사이즈
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,              # 모델 성능 향상을 위해 훈련 에포크 증가
    bf16=True,                       # A100 가속 활성화
    logging_steps=10,
    report_to="none",
    remove_unused_columns=False      # 데이터셋 컬럼 유지를 위해 필수
)

# 3. Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args               # 수정된 config 객체 전달
)

trainer.train()

# 4. 파인튜닝 후 최종 결과 확인
run_ko_test("파인튜닝 후 (Korean Fine-tuned)", ko_questions)

# 5. 파인튜닝된 어댑터 모델 저장 (한국어 모델용)
output_dir_ko = "./final_llama3_korean_adapter"
trainer.model.save_pretrained(output_dir_ko)
tokenizer.save_pretrained(output_dir_ko)
print(f"\n파인튜닝된 한국어 모델 어댑터와 토크나이저가 '{output_dir_ko}'에 저장되었습니다.")


[Step 2] Base 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


 파인튜닝 전 (Base) 테스트
테스트 1. 질문: 대한민국의 수도는 어디인가요?
결과:
질문: 대한민국의 수도는 어디인가요?
답변: 서울입니다. Seoul is the capital of South Korea.
제가 알고 있는 한국어 단어가 아닐 수 있지만, "Where is your hometown?" 라고 묻거나 할 때, 영어로 잘 대답할 방법이 없어서 질문드립니다.
저희 집은 뉴욕에 있습니다. Our home is in
--------------------------------------------------
테스트 2. 질문: 세종대왕이 만든 문자의 이름은 무엇인가요?
결과:
질문: 세종대왕이 만든 문자의 이름은 무엇인가요?
답변: 그리스도교는 유고서, 중세에 기독교를 제거하기 위해 중국에서 서양 문명을 받아들인 것이 아니라는 것을 알 수 있게 되었습니다.
--------------------------------------------------
테스트 3. 질문: 태양계에서 가장 큰 행성은 무엇인가요?
결과:
질문: 태양계에서 가장 큰 행성은 무엇인가요?
답변: 목성입니다.
태양계의 천체 중 거리순으로 열번째로 있는 목성과 지구는 비슷한 크기이고, 질량이 100배 정도 차이가 나지만 둘 사이에 존재하는 항성이 아니라서 그 차이를 생각할 필요가 없습니다. 그러나 물
--------------------------------------------------

[Step 3] 한국어 데이터 파인튜닝 시작...


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.942300
20,1.860100
30,1.825100
40,1.862300
50,1.719600
60,1.670000
70,1.708400
80,1.652700
90,1.659500



 파인튜닝 후 (Korean Fine-tuned) 테스트
테스트 1. 질문: 대한민국의 수도는 어디인가요?
결과:
질문: 대한민국의 수도는 어디인가요?
답변: 서울은 1394년에 조선왕조가 자금진에서 도읍지로 정한 도시이며, 현재 대한민국의 수도입니다. 또한 한양이라는 이름으로도 불립니다. 그러나 이명박 정부에서는 '서울'을 공식 명칭으로 사용할 것을 결정했습니다. 예전에는 서울시
--------------------------------------------------
테스트 2. 질문: 세종대왕이 만든 문자의 이름은 무엇인가요?
결과:
질문: 세종대왕이 만든 문자의 이름은 무엇인가요?
답변: 세종대왕이 만든 한글에 대한 이야기입니다. 1443년에 그는 고조선과 백제를 기반으로 하는 새로운 언어와 음운을 만들었습니다. 이때에는 몇 가지 문제가 있었는데, 첫째로 조선어는 중국인들이 사용하는 글자에서 유
--------------------------------------------------
테스트 3. 질문: 태양계에서 가장 큰 행성은 무엇인가요?
결과:
질문: 태양계에서 가장 큰 행성은 무엇인가요?
답변: 태양계의 외력행성이란, 목성보다 크고 질량이 많은 천체를 말합니다. 이 중에는 지구와 달과 같은 소행성도 포함됩니다. 이러한 천체들은 대부분 암석으로 이루어져 있으며, 태양계 내에서 다른 천체들과 충
--------------------------------------------------

파인튜닝된 한국어 모델 어댑터와 토크나이저가 './final_llama3_korean_adapter'에 저장되었습니다.
